### Feedback Ingestion into DynamoDB (using JSON)

In [13]:
import boto3
import json
import random
import time
from datetime import datetime

# Initialize DynamoDB client
dynamodb = boto3.resource('dynamodb', region_name='us-east-1')  # Change region if necessary
table = dynamodb.Table('CustomerFeedback')

# Load feedback samples from the JSON file
with open('feedback_samples.json') as f:
    feedback_samples = json.load(f)

# Function to push feedback to DynamoDB
def push_feedback_to_dynamodb(feedback):
    response = table.put_item(Item=feedback)
    print(f"Inserted feedback: {feedback['Timestamp']} for {feedback['UserID']}")
    return response

# Function to simulate feedback ingestion every 5 seconds
def ingest_feedback():
    count = 0
    while count < len(feedback_samples):
        # Select two random samples every 5 seconds
        selected_samples = random.sample(feedback_samples, 2)

        # Insert the selected feedback into DynamoDB
        for sample in selected_samples:
            # Adding the missing FeedbackID (could be generated if required, but for now, using Timestamp and UserID)
            sample['FeedbackID'] = f"{sample['Timestamp']}_{sample['UserID']}"
            push_feedback_to_dynamodb(sample)

        count += 2  # Increment count by 2 since we're inserting 2 samples at a time
        time.sleep(5)  # Wait for 5 seconds before the next batch

# Start feedback ingestion
ingest_feedback()


Inserted feedback: 2025-04-03T14:40:00Z for user33
Inserted feedback: 2025-04-03T18:55:00Z for user84
Inserted feedback: 2025-04-03T13:00:00Z for user13
Inserted feedback: 2025-04-03T13:30:00Z for user19
Inserted feedback: 2025-04-03T18:00:00Z for user73
Inserted feedback: 2025-04-03T15:50:00Z for user47
Inserted feedback: 2025-04-03T15:55:00Z for user48
Inserted feedback: 2025-04-03T15:05:00Z for user38
Inserted feedback: 2025-04-03T15:50:00Z for user47
Inserted feedback: 2025-04-03T16:00:00Z for user49
Inserted feedback: 2025-04-03T14:35:00Z for user32
Inserted feedback: 2025-04-03T12:00:00Z for user1
Inserted feedback: 2025-04-03T18:30:00Z for user79
Inserted feedback: 2025-04-03T20:05:00Z for user98
Inserted feedback: 2025-04-03T18:35:00Z for user80
Inserted feedback: 2025-04-03T15:20:00Z for user41


KeyboardInterrupt: 

### Sentiment + Score calculation via OpenAI API followed by pushing to DynamoDB

In [ ]:
import boto3
import openai
import json
import random
import time
from datetime import datetime
from decimal import Decimal  # Import Decimal

# Initialize OpenAI and DynamoDB client
openai.api_key = "key" 
dynamodb = boto3.resource('dynamodb', region_name='us-east-1') 
table = dynamodb.Table('CustomerFeedback')  # DynamoDB table name

# Function to analyze sentiment using OpenAI
def analyze_sentiment(feedback_text):
    prompt = f"Classify the sentiment of this feedback: '{feedback_text}'\nReturn the sentiment label (positive, negative, neutral) and a sentiment score between -1 and 1 based on the sentiment, in the following format: SentimentLabel: <label>, SentimentScore: <score>."

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",  
        messages=[{"role": "user", "content": prompt}],
        max_tokens=50 
    )

    sentiment_output = response.choices[0].message.content.strip()

    # Parse the sentiment label and score from the response
    try:
        sentiment_label, sentiment_score = sentiment_output.split(", ")
        sentiment_label = sentiment_label.split(":")[1].strip()
        
        # Clean up the sentiment score by removing unwanted characters (e.g., periods)
        sentiment_score_str = sentiment_score.split(":")[1].strip().rstrip('.')
        sentiment_score = float(sentiment_score_str)
        
        sentiment_score = Decimal(str(sentiment_score))  # Convert to Decimal
    except Exception as e:
        print(f"Error parsing sentiment response: {e}")
        sentiment_label = 'Neutral'
        sentiment_score = Decimal('0.0')  # Default to 0.0 as Decimal

    return sentiment_label, sentiment_score

# Function to simulate feedback and push to DynamoDB
def simulate_feedback():
    # List of hardcoded feedback samples
    feedback_samples = [
        "This trash sorting system is fantastic! It really helps me sort waste easily.",
        "The system is good, but it could be faster when identifying items.",
        "I love the idea, but the sorting process is still too complicated.",
        "The system is efficient, but it sometimes misses items.",
        "A game-changer for waste management. Great job!",
        "I think the system is useful, but there could be more instructions.",
        "The sorting experience was smooth, but some items were missed.",
        "The interface is nice, but it could be more user-friendly.",
        "I don't think this system is practical for all types of waste.",
        "The system works well, but it could use some improvements in accuracy."
    ]

    # Pick a random feedback sample
    feedback_text = random.choice(feedback_samples)
    
    # Get the current timestamp
    timestamp = datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')
    user_id = f"user{random.randint(1, 100)}"  # Simulate random user ID
    feedback_source = random.choice(['App', 'Web', 'Email'])  # Simulate random source

    # Analyze sentiment using OpenAI
    sentiment_label, sentiment_score = analyze_sentiment(feedback_text)

    # Prepare the item to be stored in DynamoDB
    feedback_item = {
        'Timestamp': timestamp,
        'UserID': user_id,
        'FeedbackSource': feedback_source,
        'FeedbackText': feedback_text,
        'SentimentLabel': sentiment_label,
        'SentimentScore': sentiment_score
    }

    # Store the feedback item in DynamoDB
    try:
        table.put_item(Item=feedback_item)
        print(f"Feedback stored successfully: {feedback_item}")
    except Exception as e:
        print(f"Error storing feedback: {e}")

# Function to run the feedback generation and store it in DynamoDB every 5 seconds
def run_feedback_simulator():
    while True:
        simulate_feedback()
        time.sleep(5)  # Wait for 5 seconds before generating the next feedback

# Main entry point
if __name__ == "__main__":
    run_feedback_simulator()


Feedback stored successfully: {'Timestamp': '2025-04-04T04:48:09Z', 'UserID': 'user74', 'FeedbackSource': 'Web', 'FeedbackText': 'I think the system is useful, but there could be more instructions.', 'SentimentLabel': 'Neutral', 'SentimentScore': Decimal('0.2')}
Feedback stored successfully: {'Timestamp': '2025-04-04T04:48:16Z', 'UserID': 'user63', 'FeedbackSource': 'Web', 'FeedbackText': 'The system works well, but it could use some improvements in accuracy.', 'SentimentLabel': 'Neutral', 'SentimentScore': Decimal('0.1')}
Feedback stored successfully: {'Timestamp': '2025-04-04T04:48:21Z', 'UserID': 'user100', 'FeedbackSource': 'Email', 'FeedbackText': 'The system is efficient, but it sometimes misses items.', 'SentimentLabel': 'neutral', 'SentimentScore': Decimal('0.2')}
Feedback stored successfully: {'Timestamp': '2025-04-04T04:48:27Z', 'UserID': 'user94', 'FeedbackSource': 'App', 'FeedbackText': 'The system is good, but it could be faster when identifying items.', 'SentimentLabel': 

KeyboardInterrupt: 

### 3 Actionable Suggestions Recommendation Engine (taking into account the past 20 latest feedback samples)

In [ ]:
import openai
import json
import boto3
from datetime import datetime

# OpenAI and DynamoDB setup
openai.api_key = "key" 
dynamodb = boto3.resource('dynamodb', region_name='us-east-1')
table = dynamodb.Table('CustomerFeedback')  # DynamoDB table name

# Function to retrieve the last N feedback entries from DynamoDB
def get_recent_feedback(n=10):
    response = table.scan()
    feedback_items = response['Items'][-n:]  # Get the last N feedback items
    return [item['FeedbackText'] for item in feedback_items]

# Function to summarize the feedback using OpenAI
def summarize_feedback(feedback_list):
    prompt = f"Summarize the following feedback and identify key areas of concern or recurring themes:\n\n"
    prompt += "\n".join(feedback_list)
    prompt += "\n\nSummary:"

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",  
        messages=[{"role": "user", "content": prompt}],
        max_tokens=1050 
    )

    summary = response.choices[0].message.content.strip()
    return summary

# Function to generate actionable suggestions using OpenAI
def generate_suggestions(summary):
    prompt = f"Based on the feedback summary below, generate 3 actionable suggestions for the marketing or product team (e.g., suggestions for improving messaging, UX, or ads to address these common issues). The suggestions should be clear, easy to implement, and focused on improving the customer experience:\n\nSummary: {summary}\n\nSuggestions:"

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",  
        messages=[{"role": "user", "content": prompt}],
        max_tokens=1050 
    )

    suggestions = response.choices[0].message.content.strip()

    return suggestions

# Function to display suggestions in a dashboard (console or web interface)
def display_suggestions(suggestions):
    print(f"Suggested Actions:\n{suggestions}\n")

# Main function to process the feedback and generate suggestions
def process_feedback():
    recent_feedback = get_recent_feedback(20)  
    summary = summarize_feedback(recent_feedback)  
    suggestions = generate_suggestions(summary)  
    display_suggestions(suggestions) 

if __name__ == "__main__":
    process_feedback()


Suggested Actions:
1. Improve interface design to enhance user-friendliness: simplify the layout, use clear icons and instructions, and provide easily accessible support resources (e.g., tutorial videos, FAQs).
2. Enhance item identification speed: optimize algorithms for quicker item recognition and provide real-time feedback to users during the scanning process to reduce frustration.
3. Expand database to include more types of waste: continuously update the system with new items to be identified, including common household items that may have been previously missed during sorting.

